In [1]:
from pathlib import Path
import utils
from typing import Callable
import scipy
from scipy import signal as sgn
import seaborn

In [2]:
def filtered_fft_features(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 8
    f_high = 40
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)

    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    return Sxx


def naif_fft_features(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    fft = np.fft.rfft(signal_matrix)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    return Sxx

def generate_df(data,features, chanels): 
    #le paso la data resultante de la extraccion de features, los nombres de los features en una lista y de 
    #los canales
    features_name=[]
    for ch in chanels:
        for feature in features:
            features_name.append(feature + 'ch ' + str (ch))
    return pd.DataFrame(data, columns=features_name)

In [3]:
class BCIDataset():
    def __init__(
        self, 
        csvs_path, 
        subject: str = 'all', 
        session: str = 'all',
        channel: str = 'all', 
        overlapping_fraction: float = 1/3, 
        window_size: int = 900,
        feature_extractor: Callable = naif_fft_features
    ):
        '''
        Object containing all examples from a time series from the dataset.
        Args:
            csvs_path (str): path al directorio donde estan los csv de los datos
            subject (str): sujeto a estudiar.
                Si se indica 'all' el dataset final generado tendrá ejemplos de todos los sujetos
            session (str): sesión a estudiar del sujeto seleccionado.
                si se indica 'all'  el dataset final generado tendrá ejemplos de todas las sesiones
            channel (str): 'ch0', 'ch1', 'ch2', 'ch3'. Si se indica 'all', los ejemplos serán la
                concatenación de los 4 canales.
            overlapping_fraction (float): porcentaje de desplazamiento de la "ventana" que hace el ejemplo
            window_size (int): tamaño de la ventana de tiempo que hace a un ejemplo (en muestras).
            feature_extractor (func): Función de extracción de features, le ingresa un arreglo
                (ejemplos en las filas, muestras en las columnas) y devuelve un arreglo (ejemplos en
                las filas y features en las columnas)
        '''
        self.csvs_path = Path(csvs_path)
        self.channel = channel
        self.parts = int(1 / overlapping_fraction)
        self.fraction = 1 / self.parts
        self.ws = window_size
        self.subject = subject
        self.session = session
        self.channels = ['ch0','ch1','ch2','ch3']
        self.feature_extractor = feature_extractor
        self.complete_dataset = utils.read_all_datasets(self.csvs_path)
        
        self.complete_examples_signal, self.complete_examples_features, \
        self.complete_labels, self.complete_metadata = \
            self.generate_examples()
        
    def generate_examples(self):
        # For each session generate the signal examples,
        # the feature extraction examples and labels arrays
        complete_examples_signal, complete_examples_features = [], []
        complete_labels, complete_metadata = [], []
        
        for subject in self.complete_dataset.subject.unique():
            # Just load the selected subject
            if self.subject != 'all' and subject != self.subject:
                continue
            sessions = \
                self.complete_dataset.loc[self.complete_dataset.subject==subject].session.unique()
            for session in sessions:
                # Just load the selected session
                if self.session != 'all' and session != self.session:
                    continue
                
                print(f'Processing subject: {subject} - session: {session}...')
                
                # Generate a subset of the dataset only with the desired rows
                selection = self.complete_dataset.loc[
                    (self.complete_dataset.subject==subject) &
                    (self.complete_dataset.session==session)
                ]
                
                # Standarize length of the array to a multiple to window size
                labels = selection.label.values
                n_rows = labels.shape[0] // self.ws
                labels = labels[: n_rows * self.ws]

                # Generate examples from the signal
                n_examples = n_rows * self.parts - (self.parts - 1)
                examples = np.empty((n_examples, self.ws))
                most_frec_labels = np.empty((n_examples, self.ws))
                times = np.empty((n_examples, self.ws))
                
                concat_ch_examples_signal, concat_ch_examples_features = [], []

                for k, ch in enumerate(self.channels):
                    # Use the four channels or just one
                    if self.channel != 'all' and self.channel != ch:
                        continue
                    
                    # Standarize length of the signal to a multiple to window size
                    signal = selection[ch].values
                    signal = signal[: n_rows * self.ws]
                    time = selection.time.values
                    time = time[: n_rows * self.ws]

                    # Increase the number of examples by overlapping the windows
                    for part in range(self.parts):
                        
                        # Find the place in the output array for each example
                        position = np.arange(part, n_examples, self.parts)
                        #position = position if part == 0 else position[:-part]

                        # Crop the signal according to the window size and overlap
                        start = int(self.ws / self.parts) * part
                        end = -int(self.ws - (self.ws / self.parts) * part)
                        end = end if part!=0 else signal.shape[0]
                        subset_signal = signal[start:end]
                        subset_labels = labels[start:end]
                        subset_times = time[start:end]

                        # Generate the examples
                        n_rows_ = int(subset_signal.shape[0]/self.ws)
                        examples[position, :] = subset_signal.reshape((n_rows_, self.ws))
                        most_frec_labels[position, :] = subset_labels.reshape((n_rows_, self.ws))
                        times[position, :] = subset_times.reshape((n_rows_, self.ws))
                    
                    # Obtain most frequent label
                    labels_ = scipy.stats.mode(most_frec_labels, axis=1).mode
                    labels_temp = scipy.stats.mode(most_frec_labels, axis=1).count
                    pureness = labels_temp == self.ws
                    # Get first and last time of the window
                    times_ = np.asarray([np.min(times, axis=1), np.max(times, axis=1)]).T
                    # Extract features
                    features = self.feature_extractor(examples)

                    concat_ch_examples_signal.append(examples.copy())
                    concat_ch_examples_features.append(features)

                concat_ch_examples_signal = np.concatenate(concat_ch_examples_signal, axis=1)
                concat_ch_examples_features = np.concatenate(concat_ch_examples_features, axis=1)

                complete_examples_signal.append(concat_ch_examples_signal)
                complete_examples_features.append(concat_ch_examples_features)
                complete_labels.append(labels_)
                lt = len(times_)
                metadata_ = np.concatenate(
                    [pureness, times_, np.repeat(subject, lt)[:,None], np.repeat(session, lt)[:,None]],
                    axis=1
                )
                complete_metadata.append(metadata_)

        complete_examples_signal = np.concatenate(complete_examples_signal)
        complete_examples_features = np.concatenate(complete_examples_features)
        complete_labels = np.concatenate(complete_labels)
        complete_metadata = np.concatenate(complete_metadata)
        
        return complete_examples_signal, complete_examples_features, complete_labels, complete_metadata
    
    def __len__(self):
        return self.complete_examples_signal.shape[0]
    
    def __getitem__(self, idx):
        return {
            'signal': self.complete_examples_signal[idx,:],
            'features': self.complete_examples_features[idx,:],
            'label': self.complete_labels[idx,:],
            'metadata': self.complete_metadata[idx,:]
        }

    def get_X_signal(self):
        return self.complete_examples_signal

    def get_X_features(self):
        return self.complete_examples_features

    def get_Y(self):
        return self.complete_labels

    def get_metadata(self):
        return self.complete_metadata

In [4]:
def calc_max_filtered(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 9
    f_high = 22
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)
    
    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    freqs = np.fft.rfftfreq(n=N, d=dt)
    
    maximum = np.argmax(Sxx, axis=1)
  
    item=freqs[maximum]
    return item.reshape((signal_matrix.shape[0], 1))

In [5]:
def filtered_range(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 9
    f_high = 22
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)

    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    freqs = np.fft.rfftfreq(n=N, d=dt)
    
    index_low=np.where(freqs>=10)[0][0]

    index_high=np.where(freqs>=35)[0][0]

    
    return Sxx[:,index_low:index_high]

In [6]:
def filtered_pca(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 10
    f_high = 35
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)

    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    freqs = np.fft.rfftfreq(n=N, d=dt)
    
    index_low=np.where(freqs>=10)[0][0]

    index_high=np.where(freqs>=35)[0][0]    
    
    from sklearn.decomposition import PCA
    pca = PCA(n_components=3)
    pca.fit(Sxx[:,index_low:index_high])
    proyected_features_freq = pca.transform(Sxx[:,index_low:index_high])
    
    return proyected_features_freq.reshape((signal_matrix.shape[0], 3))

In [7]:
csvs_path = '/home/ariel/gabriel/diplo_de_datos/mentoria_BCI/tp3'

In [8]:
# dataset con los valores de frecuencia donde aparece la amplitud máxima
dataset_filtered_max = BCIDataset(csvs_path,
                         window_size=750,
                        feature_extractor=calc_max_filtered)

Processing subject: MA - session: 1...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# dataset con la senal filtrada y sólo teniendo el rango de frecuencias de interés
dataset_filtered_range = BCIDataset(csvs_path,
                         window_size=750,
                        feature_extractor=filtered_range)

Processing subject: MA - session: 1...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# dataset con la senal filtrada y aplicando pca
dataset_filtered_pca = BCIDataset(csvs_path,
                         window_size=750,
                        feature_extractor=filtered_pca)

Processing subject: MA - session: 1...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
df_max_freq=generate_df(dataset_filtered_max.complete_examples_features,['max freq '], [0,1,2,3])

<IPython.core.display.Javascript object>

In [99]:

N=dataset_filtered_range.complete_examples_signal.shape[1]
dt = 1/200
freqs_all = np.fft.rfftfreq(n=int(N/4), d=dt)
index_low=np.where(freqs_all>=10)[0][0]
index_high=np.where(freqs_all>=35)[0][0]
name= [str(i) for i in list(freqs_all[index_low:index_high].round(2))]
df_range_freq=pd.DataFrame(dataset_filtered_range.complete_examples_features,columns=name+name+name+name)

df_range_freq['freq_label_1_mean']=np.log(df_range_freq[['12.0','12.27','12.53','12.8','13.07']].mean(axis=1))
df_range_freq['freq_label_2_mean']=np.log(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']].mean(axis=1))

df_range_freq['freq_label_1_median']=np.log(np.median(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1))
df_range_freq['freq_label_2_median']=np.log(np.median(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1))

df_range_freq['freq_label_1_sum'] = np.log(df_range_freq[['12.0','12.27','12.53','12.8','13.07']].sum(axis=1)/4)
df_range_freq['freq_label_2_sum'] = np.log(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']].sum(axis=1)/4)

df_range_freq['freq_label_1_quantile']=np.log(np.quantile(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],0.9,axis=1))
df_range_freq['freq_label_2_quantile']=np.log(np.quantile(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],0.9,axis=1))


array_freqs_label_1=df_range_freq[['12.0','12.27','12.53','12.8','13.07']].to_numpy()
array_freqs_label_2=df_range_freq[['16.0','16.27', '16.53', '16.8','17.07']].to_numpy()
#Hago una suma ponderada de los 4 canales en cjto. tener en cuenta que primero aparecen los valores 
#de cada frecuencia repetida por los cuatro canales
sum_weighted_1=np.log(np.dot(array_freqs_label_1,[0.1/4,0.1/4,0.1/4,0.1/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.4/4,0.4/4,0.4/4,0.4/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.1/4,0.1/4,0.1/4,0.1/4]))

sum_weighted_2=np.log(np.dot(array_freqs_label_2,[0.1/4,0.1/4,0.1/4,0.1/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.4/4,0.4/4,0.4/4,0.4/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.1/4,0.1/4,0.1/4,0.1/4]))
df_range_freq['sum_weighted_label_1']= sum_weighted_1.tolist()
df_range_freq['sum_weighted_label_2']= sum_weighted_2.tolist()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
df_pca_freq=generate_df(dataset_filtered_pca.complete_examples_features,['pca 0 ', 'pca 1 ','pca 2 '], [0,1,2,3])


<IPython.core.display.Javascript object>

In [14]:
labels = pd.DataFrame(dataset_filtered_max.complete_labels,columns=['labels'])
metadata_all = pd.DataFrame(dataset_filtered_max.complete_metadata, columns = ['purness', 't_start', 't_end', 
                                                                      'subject', 'session'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [138]:
df_max_freq['max_freq_mean'] = df_max_freq.mean(axis=1)
df_max_freq['max_freq_mode'] = scipy.stats.mode(df_max_freq,axis=1)[0]
df_max_freq['max_freq_median']=np.median(df_max_freq,axis=1)
df_max_freq['max_freq_quantile']=np.quantile(df_max_freq,0.9,axis=1)

max_freq_with_labels = df_max_freq.merge(labels,left_index=True,right_index=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
freq_range_with_labels = df_range_freq.merge(labels,left_index=True,right_index=True)


In [17]:
pca_with_labels = df_pca_freq.merge(labels,left_index=True,right_index=True)


In [139]:
best_features=df_max_freq.merge(df_range_freq[['freq_label_1_mean','freq_label_2_mean','freq_label_1_sum','freq_label_2_sum',
          'freq_label_1_median','freq_label_2_median','freq_label_1_quantile','freq_label_2_quantile'
                                 ,'sum_weighted_label_1','sum_weighted_label_2']],left_index=True,right_index=True)
best_features["purness"]=metadata_all["purness"].replace({'True':1,'False':0})
best_features = best_features.merge(labels,left_index=True,right_index=True)
best_features


,max freq ch 0,max freq ch 1,max freq ch 2,max freq ch 3,max_freq_mean,max_freq_mode,max_freq_median,max_freq_quantile,freq_label_1_mean,freq_label_2_mean,freq_label_1_sum,freq_label_2_sum,freq_label_1_median,freq_label_2_median,freq_label_1_quantile,freq_label_2_quantile,sum_weighted_label_1,sum_weighted_label_2,purness,labels
0,13.066667,14.933333,14.666667,13.066667,13.747619,13.066667,13.500000,14.773333,-0.829453,-0.823127,0.779985,0.786311,-1.314610,-1.352889,0.014407,0.094263,-1.150893,-0.440725,1,99.0
1,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,0.297791,0.178753,1.907229,1.788191,0.215407,0.059537,1.010042,0.853240,0.401673,0.068644,1,99.0
2,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,0.440623,0.756524,2.050061,2.365962,0.394560,0.664539,0.900520,1.341108,0.437796,0.984072,1,99.0
3,15.466667,15.466667,16.533333,16.266667,15.833333,15.466667,15.700000,16.373333,0.550711,0.869769,2.160149,2.479207,0.500371,0.809130,1.163103,1.380522,0.640049,0.988608,1,99.0
4,10.133333,18.933333,10.133333,10.133333,11.704762,10.133333,10.133333,14.596190,-0.810311,-0.663408,0.799127,0.946030,-1.001775,-0.919649,0.087729,0.110268,-0.600641,-0.537895,1,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,13.333333,11.200000,11.200000,13.600000,12.090476,11.200000,11.766667,13.440000,0.558574,0.200819,2.168012,1.810257,0.454524,-0.042428,1.009843,0.767847,0.702530,0.358629,1,1.0
2610,13.600000,19.733333,12.533333,13.866667,14.419048,12.533333,13.733333,16.544762,0.398689,0.291023,2.008127,1.900461,0.267468,0.000573,1.021108,1.062069,0.598037,0.627259,1,1.0
2611,13.866667,16.800000,13.866667,13.866667,14.390476,13.866667,13.866667,15.354286,0.136195,0.077656,1.745633,1.687094,-0.672354,-0.745241,0.945302,1.013132,0.230609,0.345996,1,1.0
2612,12.533333,15.733333,12.533333,18.666667,14.366667,12.533333,13.700000,16.906667,0.109989,-0.164589,1.719427,1.444849,-0.891365,-0.354435,1.136907,0.461768,0.579868,-0.294638,1,1.0


In [140]:
X, y = best_features.iloc[:, :-1], best_features.labels

In [141]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y) # se queda con el 10% para validación
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train) # del 90% hace 20% para train y 80% para test

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [142]:
y_train.value_counts()

99.0    1158
2.0      377
1.0      346
Name: labels, dtype: int64

In [78]:
X

,max freq ch 0,max freq ch 1,max freq ch 2,max freq ch 3,max_freq_mean,max_freq_mode,max_freq_median,freq_label_1_mean,freq_label_2_mean,freq_label_1_sum,freq_label_2_sum,freq_label_1_median,freq_label_2_median,freq_label_1_quantile,freq_label_2_quantile,sum_weighted_label_1,sum_weighted_label_2,purness
0,13.066667,14.933333,14.666667,13.066667,13.933333,13.066667,13.500000,0.436288,0.439057,2.181440,2.195283,0.268579,0.258492,0.014407,0.094263,0.316354,0.643570,1
1,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,1.346880,1.195726,6.734401,5.978628,1.240366,1.061345,1.010042,0.853240,1.494322,1.071055,1
2,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,1.553675,2.130857,7.768376,10.654286,1.483732,1.943594,0.900520,1.341108,1.549289,2.675327,1
3,15.466667,15.466667,16.533333,16.266667,15.933333,15.466667,15.700000,1.734486,2.386360,8.672431,11.931802,1.649332,2.245953,1.163103,1.380522,1.896574,2.687492,1
4,10.133333,18.933333,10.133333,10.133333,12.333333,10.133333,10.133333,0.444720,0.515093,2.223599,2.575466,0.367227,0.398659,0.087729,0.110268,0.548460,0.583976,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,13.333333,11.200000,11.200000,13.600000,12.333333,11.200000,11.766667,1.748178,1.222403,8.740891,6.112017,1.575423,0.958460,1.009843,0.767847,2.018854,1.431366,1
2610,13.600000,19.733333,12.533333,13.866667,14.933333,12.533333,13.733333,1.489870,1.337795,7.449348,6.688976,1.306652,1.000573,1.021108,1.062069,1.818546,1.872472,1
2611,13.866667,16.800000,13.866667,13.866667,14.600000,13.866667,13.866667,1.145906,1.080751,5.729528,5.403754,0.510505,0.474620,0.945302,1.013132,1.259367,1.413397,1
2612,12.533333,15.733333,12.533333,18.666667,14.866667,12.533333,13.700000,1.116265,0.848243,5.581327,4.241213,0.410096,0.701570,1.136907,0.461768,1.785803,0.744801,1


In [143]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
target_names = ['1', '2','99']


dc = DummyClassifier(strategy='uniform')
dc.fit(X_train, y_train)
y_train_pred=dc.predict(X_train)
print(classification_report(y_train, y_train_pred ))

              precision    recall  f1-score   support

         1.0       0.17      0.29      0.21       346
         2.0       0.20      0.37      0.26       377
        99.0       0.62      0.32      0.42      1158

    accuracy                           0.33      1881
   macro avg       0.33      0.33      0.30      1881
weighted avg       0.46      0.33      0.35      1881



In [105]:
y_val_pred = dc.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

         1.0       0.18      0.35      0.24        48
         2.0       0.17      0.28      0.21        53
        99.0       0.57      0.29      0.38       161

    accuracy                           0.30       262
   macro avg       0.31      0.31      0.28       262
weighted avg       0.42      0.30      0.32       262



In [106]:
y_test_pred = dc.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         1.0       0.18      0.29      0.22        87
         2.0       0.20      0.31      0.24        94
        99.0       0.61      0.39      0.47       290

    accuracy                           0.35       471
   macro avg       0.33      0.33      0.31       471
weighted avg       0.45      0.35      0.38       471



In [144]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

OVR=OneVsRestClassifier(LinearSVC(random_state=0))
OVR.fit(X_train, y_train)
y_train_pred = OVR.predict(X_train)
y_val_pred = OVR.predict(X_val)

/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [145]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_train_pred))


              precision    recall  f1-score   support

         1.0       0.34      0.82      0.48       346
         2.0       0.48      0.50      0.49       377
        99.0       0.80      0.46      0.58      1158

    accuracy                           0.53      1881
   macro avg       0.54      0.59      0.52      1881
weighted avg       0.65      0.53      0.54      1881



In [146]:
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

         1.0       0.33      0.79      0.47        48
         2.0       0.49      0.58      0.53        53
        99.0       0.81      0.42      0.56       161

    accuracy                           0.52       262
   macro avg       0.54      0.60      0.52       262
weighted avg       0.66      0.52      0.53       262



In [147]:
y_test_pred = OVR.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         1.0       0.35      0.87      0.50        87
         2.0       0.48      0.51      0.49        94
        99.0       0.79      0.41      0.54       290

    accuracy                           0.52       471
   macro avg       0.54      0.60      0.51       471
weighted avg       0.65      0.52      0.53       471



from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
cm=confusion_matrix(y_test,y_test_pred)
print('matriz de confusion test')
print(cm)
#tn, fp, fn, tp = confusion_matrix(y_test,y_test_pred).ravel()
print('matriz de confusion test')
plot_confusion_matrix(OVR.fit(X_train, y_train), X_test, y_test,
                                 display_labels=target_names,
                                 cmap=plt.cm.Blues,
                                 normalize='true')

In [148]:
from sklearn.multiclass import OneVsOneClassifier
OVO=OneVsOneClassifier(LinearSVC(random_state=0))
OVO.fit(X_train, y_train)
y_train_pred = OVO.predict(X_train)
y_val_pred = OVO.predict(X_val)

/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [149]:
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

         1.0       0.73      0.23      0.35        48
         2.0       0.67      0.15      0.25        53
        99.0       0.65      0.95      0.77       161

    accuracy                           0.66       262
   macro avg       0.68      0.44      0.46       262
weighted avg       0.67      0.66      0.59       262



In [150]:
y_test_pred = OVO.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         1.0       0.77      0.39      0.52        87
         2.0       0.68      0.16      0.26        94
        99.0       0.68      0.95      0.79       290

    accuracy                           0.69       471
   macro avg       0.71      0.50      0.52       471
weighted avg       0.70      0.69      0.63       471



### Random forest

In [151]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
param = {
    'n_estimators':[100,150,200],
    'criterion':['gini','entropy'],
    #'max_depth':[25,20,None,10],
    'min_samples_split':[2,5,None]
}
param_grid = {
  'n_estimators': [150],
  'max_features': [5, 7, 9],
  'max_depth'   : [None, 3, 10, 20],
  'criterion'   : ['gini', 'entropy']
}
cv = GridSearchCV(estimator=rfc,param_grid=param, scoring='f1_micro',n_jobs=-1,cv=5, verbose=1)
cv.fit(X_train, y_train)
y_train_pred = cv.predict(X_train)
y_val_pred = cv.predict(X_val)



<IPython.core.display.Javascript object>

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   49.0s finished


In [115]:
best_tree_clf = cv.best_estimator_
best_tree_clf

RandomForestClassifier(min_samples_split=5, n_estimators=150)

In [152]:
print(classification_report(y_train, y_train_pred))


              precision    recall  f1-score   support

         1.0       0.99      0.99      0.99       346
         2.0       1.00      0.98      0.99       377
        99.0       0.99      1.00      1.00      1158

    accuracy                           0.99      1881
   macro avg       1.00      0.99      0.99      1881
weighted avg       0.99      0.99      0.99      1881



In [153]:

print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

         1.0       0.77      0.35      0.49        48
         2.0       0.61      0.32      0.42        53
        99.0       0.69      0.91      0.79       161

    accuracy                           0.69       262
   macro avg       0.69      0.53      0.56       262
weighted avg       0.69      0.69      0.66       262



In [154]:
y_test_pred = cv.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         1.0       0.79      0.44      0.56        87
         2.0       0.64      0.37      0.47        94
        99.0       0.71      0.91      0.80       290

    accuracy                           0.71       471
   macro avg       0.71      0.57      0.61       471
weighted avg       0.71      0.71      0.69       471



In [155]:
cv.best_params_

{'criterion': 'gini', 'min_samples_split': 5, 'n_estimators': 150}

### Support vector machine

In [156]:
from sklearn.svm import SVC

svc_model = SVC(gamma='auto',random_state=100)
svc_model.fit(X_train, y_train)

y_train_pred = svc_model.predict(X_train)
y_val_pred = svc_model.predict(X_val)
y_test_pred = svc_model.predict(X_test)


In [157]:
print(classification_report(y_train, y_train_pred))


              precision    recall  f1-score   support

         1.0       0.76      0.37      0.49       346
         2.0       0.81      0.31      0.45       377
        99.0       0.70      0.95      0.81      1158

    accuracy                           0.72      1881
   macro avg       0.75      0.54      0.58      1881
weighted avg       0.73      0.72      0.68      1881



In [158]:
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

         1.0       0.65      0.27      0.38        48
         2.0       0.64      0.26      0.37        53
        99.0       0.67      0.92      0.78       161

    accuracy                           0.67       262
   macro avg       0.65      0.48      0.51       262
weighted avg       0.66      0.67      0.62       262



In [159]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         1.0       0.76      0.39      0.52        87
         2.0       0.71      0.27      0.39        94
        99.0       0.69      0.93      0.79       290

    accuracy                           0.70       471
   macro avg       0.72      0.53      0.56       471
weighted avg       0.71      0.70      0.66       471



### gradient boosting

In [160]:

from sklearn.ensemble import GradientBoostingClassifier
# GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=100)
gbc.fit(X_train, y_train)
y_train_pred = gbc.predict(X_train)
y_val_pred = gbc.predict(X_val)
y_test_pred = gbc.predict(X_test)


In [161]:
print(classification_report(y_train, y_train_pred))


              precision    recall  f1-score   support

         1.0       0.81      0.63      0.71       346
         2.0       0.90      0.55      0.69       377
        99.0       0.80      0.95      0.87      1158

    accuracy                           0.81      1881
   macro avg       0.84      0.71      0.75      1881
weighted avg       0.82      0.81      0.80      1881



In [162]:
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

         1.0       0.74      0.42      0.53        48
         2.0       0.62      0.40      0.48        53
        99.0       0.73      0.91      0.81       161

    accuracy                           0.71       262
   macro avg       0.69      0.57      0.61       262
weighted avg       0.71      0.71      0.69       262



In [163]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         1.0       0.73      0.49      0.59        87
         2.0       0.71      0.38      0.50        94
        99.0       0.73      0.91      0.81       290

    accuracy                           0.73       471
   macro avg       0.72      0.59      0.63       471
weighted avg       0.72      0.73      0.71       471



### SGDC

In [164]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
sgdc = SGDClassifier(random_state=100)
sgdc.fit(X_train, y_train)
y_train_pred = sgdc.predict(X_train)
y_val_pred = sgdc.predict(X_val)
y_test_pred = sgdc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       0.91      0.06      0.11       346
         2.0       0.51      0.29      0.37       377
        99.0       0.65      0.92      0.76      1158

    accuracy                           0.64      1881
   macro avg       0.69      0.42      0.41      1881
weighted avg       0.67      0.64      0.56      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       1.00      0.08      0.15        48
         2.0       0.49      0.32      0.39        53
        99.0       0.65      0.90      0.76       161

    accuracy                           0.63       262
   macro avg       0.71      0.43      0.43       262
weighted avg       0.68      0.63      0.57       262

reporte test
              precision    recall  f1-score   support

         1.0       1.00      0.10      0.19        87
         2.0       0.56      0.33      0.42        94
        99.0       0.66      

### logistic regresion

In [165]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=100)
lrc.fit(X_train, y_train)
y_train_pred = lrc.predict(X_train)
y_val_pred = lrc.predict(X_val)
y_test_pred = lrc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       0.67      0.36      0.47       346
         2.0       0.60      0.27      0.38       377
        99.0       0.69      0.90      0.78      1158

    accuracy                           0.68      1881
   macro avg       0.65      0.51      0.54      1881
weighted avg       0.67      0.68      0.64      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.67      0.33      0.44        48
         2.0       0.61      0.32      0.42        53
        99.0       0.68      0.88      0.77       161

    accuracy                           0.67       262
   macro avg       0.65      0.51      0.54       262
weighted avg       0.66      0.67      0.64       262

reporte test
              precision    recall  f1-score   support

         1.0       0.73      0.53      0.61        87
         2.0       0.69      0.35      0.46        94
        99.0       0.72      

/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### decision tree

In [166]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=100)
dtc.fit(X_train, y_train)
y_train_pred = dtc.predict(X_train)
y_val_pred = dtc.predict(X_val)
y_test_pred = dtc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       346
         2.0       1.00      1.00      1.00       377
        99.0       1.00      1.00      1.00      1158

    accuracy                           1.00      1881
   macro avg       1.00      1.00      1.00      1881
weighted avg       1.00      1.00      1.00      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.42      0.38      0.40        48
         2.0       0.40      0.47      0.43        53
        99.0       0.73      0.71      0.72       161

    accuracy                           0.60       262
   macro avg       0.52      0.52      0.52       262
weighted avg       0.61      0.60      0.60       262

reporte test
              precision    recall  f1-score   support

         1.0       0.51      0.48      0.50        87
         2.0       0.46      0.48      0.47        94
        99.0       0.72      